In [ ]:
#



# Comparative Performance Analysis of Gradient Boosting, XGBoost, and CatBoost on Cancer dataset . 
# Boosting means combining multiple weak learners to create a strong learner.and correcting errors of previous models.



#

In [ ]:
pip install xgboost catboost scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split #80% train, 20% test, labels for training and testing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#accuracy_score - It tells you how many predictions were correct.
#confusion_matrix - It shows how many were predicted correctly and incorrectly for each class.[tp,tn,fp,fn]
#classification_report -It gives precision    recall  f1-score   support

In [ ]:
file_path = r"C:\Users\shara\OneDrive\Desktop\ML ACTS\ML-Projects\XG,CAT,GRAD Boosting project 4\boosting_dataset.csv"
df = pd.read_csv(file_path)

df = df.drop(['id'], axis=1) #id no use 
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0}) # in dataset M=malignant=1, B=benign=0 => label encoded 1,0
X = df.drop('diagnosis', axis=1) # features
y = df['diagnosis'] # target variable

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( 
    X, y, test_size=0.2, random_state=42 #random_state=42 is convention(any no. will work), it acts as a seed for the random number generator. 
)
  # This means that every time you run your code, the random number generator will Always split exactly the same here, ensuring same results each time.

In [ ]:
# Scale features as large numbers dominate — model becomes biased.
scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train) #mean deviation and transform -> z=(x-mean)/std
X_test = scaler.transform(X_test) # not fit to avoid DATA LEAKAGE 
#If your model looks at test data during training, it “cheats” and learns patterns it should not know.
#fit = learn  , transform = apply

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=150, learning_rate=0.1) #n_estimators = trees = weak learners
# excessive trees -> overfitting , 
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)#output like :[1, 0, 1, 1, 0, ...]

print(confusion_matrix(y_test, y_pred_gb))
print("\n\n\n",classification_report(y_test, y_pred_gb))

[[69  2]
 [ 3 40]]



               precision    recall  f1-score   support

           0       0.96      0.97      0.97        71
           1       0.95      0.93      0.94        43

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114



In [ ]:
import xgboost as xgb     #vimp
xgb_model = xgb.XGBClassifier(
    n_estimators=150,
    learning_rate=0.1,
    eval_metric='logloss' #Log loss measures: how well the model’s predicted probability matches the true label.
)# log loss as we are doing classification and you want probabilities 0.93(logloss extremely small), 0.10(logloss huge)
# other options than logloss - error(0/1 classif), auc , rmse and mae(reg), hinge(SVM), merror and mlogloss(multi-class), quantile(outliers)
#for multiple loss func in xg , we can write eval_metric=['logloss', 'auc'] etc
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

print(confusion_matrix(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))

[[69  2]
 [ 3 40]]
              precision    recall  f1-score   support

           0       0.96      0.97      0.97        71
           1       0.95      0.93      0.94        43

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114



In [ ]:
from catboost import CatBoostClassifier  #for categorical data handling (category+boosting)
cat = CatBoostClassifier(
    iterations=150,
    learning_rate=0.1,
    verbose=0   #verbose controls how much CatBoost prints during training: 
                 #verbose=0 silences all messages, verbose=10 prints current loss every 10 iterations;
)#useful to avoid flooding the console on large datasets or many trees, and it does not affect model performance.

cat.fit(X_train, y_train)
y_pred_cat = cat.predict(X_test)

print(confusion_matrix(y_test, y_pred_cat))
print(classification_report(y_test, y_pred_cat))

#output confusion matrics 
#[[70  1]   → Actual 0 (Benign)
# [ 2 41]]  → Actual 1 (Malignant)
#TN = 70 → correctly predicted Benign  FP = 1 → predicted Malignant but actually Benign 
#FN = 2 → predicted Benign but actually Malignant  TP = 41 → correctly predicted Malignant

[[70  1]
 [ 2 41]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98        71
           1       0.98      0.95      0.96        43

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114



In [ ]:
#comparing accuracies of all three models
print("Gradient Boosting:", accuracy_score(y_test, y_pred_gb))
print("XGBoost:", accuracy_score(y_test, y_pred_xgb))
print("CatBoost:", accuracy_score(y_test, y_pred_cat))

Gradient Boosting: 0.956140350877193
XGBoost: 0.956140350877193
CatBoost: 0.9736842105263158


In [ ]:
#accuracy_score - It tells you how many predictions were correct.
#confusion_matrix - It shows how many were predicted correctly and incorrectly for each class.[tp,tn,fp,fn]
#classification_report -It gives precision    recall  f1-score   support


#If your model looks at test data during training, it “cheats” and learns patterns it should not know.
#fit = learn  , transform = apply



#output confusion matrics 
#[[70  1]   → Actual 0 (Benign)
# [ 2 41]]  → Actual 1 (Malignant)
#TN = 70 → correctly predicted Benign  FP = 1 → predicted Malignant but actually Benign 
#FN = 2 → predicted Benign but actually Malignant  TP = 41 → correctly predicted Malignant



# other options than logloss - error(0/1 classif), auc , rmse and mae(reg), hinge(SVM), merror and mlogloss(multi-class), quantile(outliers)
#for multiple loss func in xg , we can write eval_metric=['logloss', 'auc'] etc


                 #verbose=0 silences all messages, verbose=10 prints current loss every 10 iterations;
  # This means that every time you run your code, the random number generator will Always split exactly the same here, ensuring same results each time.

#useful to avoid flooding the console on large datasets or many trees, and it does not affect model performance.


# log loss as we are doing classification and you want probabilities 0.93(logloss extremely small), 0.10(logloss huge)
